<a href="https://colab.research.google.com/github/ggwnsghgg/Exchange_rate_prediction_Project/blob/main/LSTM_%EC%98%88%EC%8B%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
filename = '/content/drive/My Drive/test.csv'

In [5]:
data = pd.read_csv(filename)
data.head()

,Date,ExchangeRate,BID,OFFER,KORIBOR,LIBOR,Dollar,Call,KOSPI
0,2018.02.26,"1,078.50",-1370,-870,1.99,1.4475,89.85,1.48,"2,457.65"
1,2018.02.27,"1,073.70",-1390,-890,2.00,1.4475,90.36,1.50,"2,456.14"
2,2018.02.28,"1,071.00",-1400,-900,2.00,1.4425,90.61,1.51,"2,427.36"
3,2018.03.02,"1,081.90",-1400,-900,2.00,1.4475,89.94,1.50,"2,402.16"
4,2018.03.05,"1,081.30",-1360,-860,2.01,1.4475,90.08,1.46,"2,375.06"


In [6]:
# 결측치 제거
dataset = data.dropna()

In [7]:
data

,Date,ExchangeRate,BID,OFFER,KORIBOR,LIBOR,Dollar,Call,KOSPI
0,2018.02.26,"1,078.50",-1370,-870,1.99,1.4475,89.85,1.48,"2,457.65"
1,2018.02.27,"1,073.70",-1390,-890,2.00,1.4475,90.36,1.50,"2,456.14"
2,2018.02.28,"1,071.00",-1400,-900,2.00,1.4425,90.61,1.51,"2,427.36"
3,2018.03.02,"1,081.90",-1400,-900,2.00,1.4475,89.94,1.50,"2,402.16"
4,2018.03.05,"1,081.30",-1360,-860,2.01,1.4475,90.08,1.46,"2,375.06"
...,...,...,...,...,...,...,...,...,...
734,2021.02.17,"1,099.70",-315,155,0.87,NaN,90.95,0.47,"3,133.73"
735,2021.02.18,"1,107.70",-305,165,0.87,NaN,90.59,0.49,"3,086.66"
736,2021.02.19,"1,105.70",-305,165,0.86,NaN,90.36,0.51,"3,107.62"
737,2021.02.22,"1,107.10",-305,165,0.86,NaN,90.01,0.50,"3,079.75"


,대미환산율
count,4327.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0
